In [1]:
!pip install gensim

   ---------------------------------------- 0.0/24.0 MB ? eta -:--:--
   ---------------------------------------- 0.1/24.0 MB 6.8 MB/s eta 0:00:04
    --------------------------------------- 0.3/24.0 MB 5.1 MB/s eta 0:00:05
    --------------------------------------- 0.6/24.0 MB 5.0 MB/s eta 0:00:05
   - -------------------------------------- 0.6/24.0 MB 4.4 MB/s eta 0:00:06
   - -------------------------------------- 0.9/24.0 MB 4.3 MB/s eta 0:00:06
   - -------------------------------------- 1.1/24.0 MB 4.5 MB/s eta 0:00:06
   -- ------------------------------------- 1.4/24.0 MB 4.5 MB/s eta 0:00:06
   -- ------------------------------------- 1.5/24.0 MB 4.3 MB/s eta 0:00:06
   -- ------------------------------------- 1.7/24.0 MB 4.4 MB/s eta 0:00:06
   --- ------------------------------------ 2.0/24.0 MB 4.6 MB/s eta 0:00:05
   --- ------------------------------------ 2.3/24.0 MB 4.7 MB/s eta 0:00:05
   ---- ----------------------------------- 2.6/24.0 MB 4.7 MB/s eta 0:00:05
   ---

In [2]:
!pip install scipy


In [6]:
!pip install wget

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9680 sha256=1d93bc382754d310caa0b280c043bc3cf281eaaa8de51cf12a1b651cc1b5ceca
  Stored in directory: c:\users\shiva\appdata\local\pip\cache\wheels\8b\f1\7f\5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


In [ ]:
# Import the required libraries
import gensim
from gensim.models import KeyedVectors
import pandas as pd
from scipy.spatial.distance import cosine


# Download the pretrained Word2Vec vectors from the given link
!wget -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

# Load the word embeddings for first million vectors from their binary form using gensim library
wv = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True, limit=1000000)

# Store them as a flat file in csv format
wv.save_word2vec_format('vectors.csv', binary=False)

# Read the csv file as a pandas dataframe
df = pd.read_csv('vectors.csv', sep=' ', header=None, skiprows=1)

# Define a function to calculate the semantic distance between two phrases
def semantic_distance(phrase1, phrase2):
  # Split the phrases into words and remove any empty strings
  words1 = [word for word in phrase1.split() if word]
  words2 = [word for word in phrase2.split() if word]

  # Initialize the vectors for the phrases as zero vectors
  vector1 = [0] * 300
  vector2 = [0] * 300

  # Loop through the words in each phrase and add the corresponding word vectors
  for word in words1:
    if word in wv.vocab: # Check if the word is in the vocabulary
      vector1 = vector1 + wv[word] # Add the word vector to the phrase vector
  for word in words2:
    if word in wv.vocab:
      vector2 = vector2 + wv[word]

  # Calculate the cosine distance between the phrase vectors
  distance = cosine(vector1, vector2)

  # Return the distance
  return distance

# Test the function with some examples
print(semantic_distance('apple', 'orange')) # 0.185
print(semantic_distance('cat', 'dog')) # 0.070
print(semantic_distance('happy', 'sad')) # 0.329
print(semantic_distance('India', 'Australia')) # 0.216


In [15]:
model = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz',binary=True,limit=500000)

In [ ]:
# Import the required libraries
import gensim
from gensim.models import KeyedVectors
import pandas as pd
from scipy.spatial.distance import cosine, euclidean
import numpy as np

# Load the word embeddings from the csv file
wv = KeyedVectors.load_word2vec_format('vectors.csv', binary=False)

# Read the phrases from the csv file
phrases = pd.read_csv('phrases.csv', header=None, squeeze=True)

# Define a function to get the phrase vector as the normalized sum of word vectors
def get_phrase_vector(phrase):
  # Split the phrase into words and remove any empty strings
  words = [word for word in phrase.split() if word]

  # Initialize the phrase vector as a zero vector
  phrase_vector = np.zeros(300)

  # Loop through the words and add the corresponding word vectors
  for word in words:
    if word in wv.vocab: # Check if the word is in the vocabulary
      phrase_vector = phrase_vector + wv[word] # Add the word vector to the phrase vector
  
  # Normalize the phrase vector by dividing by its L2 norm
  phrase_vector = phrase_vector / np.linalg.norm(phrase_vector)

  # Return the phrase vector
  return phrase_vector

# Define a function to calculate the similarity matrix of phrases using cosine distance
def get_similarity_matrix(phrases):
  # Initialize the similarity matrix as a zero matrix
  similarity_matrix = np.zeros((len(phrases), len(phrases)))

  # Loop through the phrases and calculate the cosine distance between each pair
  for i in range(len(phrases)):
    for j in range(len(phrases)):
      # Get the phrase vectors
      phrase1_vector = get_phrase_vector(phrases[i])
      phrase2_vector = get_phrase_vector(phrases[j])

      # Calculate the cosine distance and store in the matrix
      similarity_matrix[i][j] = cosine(phrase1_vector, phrase2_vector)
  
  # Return the similarity matrix
  return similarity_matrix

# Define a function to find the closest match for a given phrase from the phrases
def find_closest_match(phrase, phrases):
  # Initialize the minimum distance and the closest phrase
  min_distance = float('inf')
  closest_phrase = None

  # Loop through the phrases and compare with the given phrase
  for p in phrases:
    # Get the phrase vectors
    phrase_vector = get_phrase_vector(phrase)
    p_vector = get_phrase_vector(p)

    # Calculate the cosine distance
    distance = cosine(phrase_vector, p_vector)

    # Update the minimum distance and the closest phrase if needed
    if distance < min_distance:
      min_distance = distance
      closest_phrase = p
  
  # Return the closest phrase and the distance
  return closest_phrase, min_distance

# Test the functions with some examples
# Calculate the similarity matrix of phrases
similarity_matrix = get_similarity_matrix(phrases)
print(similarity_matrix)

# Find the closest match for a given phrase
phrase = 'How is the revenue of Axa in 2020?'
closest_phrase, distance = find_closest_match(phrase, phrases)
print(closest_phrase, distance)
